In [1]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [57]:
from sklearn.preprocessing import MultiLabelBinarizer

In [62]:
from sklearn.model_selection import train_test_split

In [3]:
t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.12357783317565918  seconds 


In [4]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [5]:


def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object



In [6]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [7]:
def read_xml(li):
    
    
    all_objects=[]
    for i in range(len(li)):
    # print("i will run ", i+1," times")
        name_of_file=clean_file_names(li[i])
    # print(name_of_file)
        data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

        post_text='  '
        tree = etree.parse(li[i], parser=parser)
        root = tree.getroot()
        for element in root.iter():
            if element.tag == 'post':
                element_text = element.text
                post_text = post_text + element_text
        mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
        mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



        data_object['posts']=mystring   
        all_objects.append(data_object)

    return all_objects

In [8]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [11]:
df=pd.DataFrame(data)

In [20]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(posts):
    filtered_post=[]
    
    word_tokens = word_tokenize(posts) 
    filtered_sentence = []   
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    filtered_post.append(filtered_sentence)
    return filtered_post

In [38]:
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered    

In [48]:
for i in range(len(data)):
    data[i]['cleaned_stop_words']=fun_remove_stop_words(data[i]['posts'])

In [49]:
def remove_dashes(data):
    for i in range(len(data)):
        data[i]['cleaned_stop_words']=data[i]['cleaned_stop_words'].replace('--', ' ')
        

In [50]:
remove_dashes(data)

In [51]:
data[1]['cleaned_stop_words']

"  love studying?can't get enough fat books manic profs.?  well, rare species, need worry, This dedicated uncle, died duty age 29, police work... The Day I Cried                  â\x80\x9cThose I fight I hate,Those I guard I love;â\x80\x9d  - (An Irish Airman Foresees His Death, W.B. Yeats)  It white room, The corpse winding sheet, My aunt new white sari, Staring silence gone. I could hear leaves rustle outside window, But I didnâ\x80\x99t cry.  She gone crazy, mom whispered, Between sobs, â\x80\x9cShe 25, poor girlâ\x80\x9d I continued look down, At mosaic floor, registering word. I could hear white saris softly flapping, But I couldnâ\x80\x99t cry.  My cousin sister came me, Dressed prettiest frock occasion. She five, understand? I took buy chocolate. I could hear neighborsâ\x80\x99 shushing went by, But I still didnâ\x80\x99t cry.  On way back train, Three passengers discussing police, â\x80\x9cAll corrupt sons bitches â\x80\x93 deserve dieâ\x80\x9d I remembered you, uncle, proud un

In [52]:
df=pd.DataFrame(data)

In [53]:
X=df['cleaned_stop_words']

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [56]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [58]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [59]:
labels = mlb.fit_transform(labels)

In [60]:
y=labels

In [64]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, random_state=42)

In [65]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [67]:


input_dim = trainX.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [68]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                16218700  
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 14)                14350     
Total params: 16,248,620
Trainable params: 16,

In [ ]:
trainX, testX, trainY, testY
classifier_nn = model.fit(trainX,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(testX, testY),
                    batch_size=10)






Train on 11680 samples, validate on 2920 samples
Epoch 1/5





11680/11680 [==============================] - 272s 23ms/step - loss: 0.3841 - acc: 0.8576 - val_loss: 0.4962 - val_acc: 0.8641
Epoch 2/5
11680/11680 [==============================] - 262s 22ms/step - loss: 0.3784 - acc: 0.8606 - val_loss: 0.4847 - val_acc: 0.8583
Epoch 3/5
 1350/11680 [==>...........................] - ETA: 3:27 - loss: 0.3741 - acc: 0.8669

In [ ]:
model.predict(trainX[33])

In [ ]:
trainY[33]

In [ ]:
print("time taken to read :: ", (t2-t1)/100)

In [153]:
data_backup=data.copy()

### Word2 Vector

In [25]:
words_of_posts=[]
for j in range(len(data)):
    posts=data[j]['posts']
    
    words=[]
    for i in range(len(posts)):
        words.append(word_tokenize(posts[i]))
    words_of_posts.append(words)

KeyboardInterrupt: 

In [21]:
remove_stop_words(data[1]['posts'])

[['love',
  'studying',
  '?',
  'ca',
  "n't",
  'get',
  'enough',
  'fat',
  'books',
  'manic',
  'profs.',
  '?',
  'well',
  ',',
  'rare',
  'species',
  ',',
  'need',
  'worry',
  ',',
  'This',
  'dedicated',
  'uncle',
  ',',
  'died',
  'duty',
  'age',
  '29',
  ',',
  'police',
  'work',
  '...',
  'The',
  'Day',
  'I',
  'Cried',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  'â\x80\x9cThose',
  'I',
  'fight',
  'I',
  'hate',
  ',',
  'Those',
  'I',
  'guard',
  'I',
  'love',
  ';',
  'â\x80\x9d',
  '-',
  '(',
  'An',
  'Irish',
  'Airman',
  'Foresees',
  'His',
  'Death',
  ',',
  'W.B',
  '.',
  'Yeats',
  ')',
  'It',
  'white',
  'room',
  ',',
  'The',
  'corpse',
  'winding',
  'sheet',
  ',',
  'My',
  'aunt',
  'new',
  'white',
  'sari',
  ',',
  'Staring',
  'silence',
  'gone',
  '.',
  'I',
  'could',
  'hear',
  'leaves',
  'rustle',
  'outside',
  'wind

In [ ]:
if len(data)==len(words_of_posts):
    for i in range(len(data)):
        data[i]['tokenized']=words_of_posts[i]

In [ ]:
df=pd.DataFrame(data)

In [ ]:
X=df['tokenized'].values

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
y=data['horoscope']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

In [ ]:
y_transform=le.transform(y)

In [ ]:
from keras.utils import np_utils

In [ ]:
dummy_y = np_utils.to_categorical(y_transform)

In [ ]:
#Split

In [ ]:
# model

In [ ]:
# compile

In [ ]:
# fit

In [113]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [121]:
freq_matrix=_create_frequency_matrix(data[1]['posts'])

In [115]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [123]:
tf_matrix=_create_tf_matrix(freq_matrix)

In [117]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [125]:
doc_per_word=_create_documents_per_words(freq_matrix)

In [119]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [129]:
_create_idf_matrix(freq_matrix,doc_per_word,total_documents)

{' love studying?': {'love': 0.8129133566428556,
  'studi': 1.1139433523068367,
  '?': 0.8129133566428556,
  'ca': 1.1139433523068367,
  "n't": 1.1139433523068367,
  'get': 1.1139433523068367,
  'enough': 1.1139433523068367,
  'fat': 1.1139433523068367,
  'book': 1.1139433523068367,
  'manic': 1.1139433523068367,
  'prof': 1.1139433523068367,
  '.': 0.15970084286751188},
 'well, you are a': {'well': 1.1139433523068367,
  ',': 0.15970084286751188,
  'rare': 1.1139433523068367,
  'speci': 1.1139433523068367,
  'need': 1.1139433523068367,
  'worri': 1.1139433523068367,
  'thi': 1.1139433523068367,
  'dedic': 1.1139433523068367,
  'uncl': 0.8129133566428556,
  'die': 1.1139433523068367,
  'duti': 1.1139433523068367,
  'age': 1.1139433523068367,
  '29': 1.1139433523068367,
  'polic': 0.8129133566428556,
  'work': 1.1139433523068367,
  '...': 1.1139433523068367},
 'The Day I Cried': {'day': 1.1139433523068367,
  'criedâ\x80\x9cthos': 1.1139433523068367,
  'fight': 1.1139433523068367,
  'hate